In [21]:
from sklearn.datasets import make_regression
from lightgbm import LGBMRegressor
import pandas as pd
import re

## LightGBM

In [6]:
# Fitting a model
X, y = make_regression()
lgb_reg = LGBMRegressor().fit(X, y)

In [7]:
lgb_reg.booster_.trees_to_dataframe()

,tree_index,node_depth,node_index,left_child,right_child,parent_index,split_feature,split_gain,threshold,decision_type,missing_direction,missing_type,value,weight,count
0,0,1,0-S0,0-L0,0-S1,None,Column_65,911502.00000,-0.291331,<=,left,None,-1.896870,0,100
1,0,2,0-L0,None,None,0-S0,None,NaN,NaN,None,None,None,-14.626549,36,36
2,0,2,0-S1,0-L1,0-L2,0-S0,Column_14,486131.00000,0.101904,<=,left,None,5.263580,64,64
3,0,3,0-L1,None,None,0-S1,None,NaN,NaN,None,None,None,-3.728552,31,31
4,0,3,0-L2,None,None,0-S1,None,NaN,NaN,None,None,None,13.710735,33,33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
683,99,3,99-S2,99-L0,99-L3,99-S1,Column_72,1836.77002,-0.340918,<=,left,None,-0.818543,41,41
684,99,4,99-L0,None,None,99-S2,None,NaN,NaN,None,None,None,-1.504394,20,20
685,99,4,99-L3,None,None,99-S2,None,NaN,NaN,None,None,None,-0.165351,21,21
686,99,3,99-L2,None,None,99-S1,None,NaN,NaN,None,None,None,0.386470,36,36


## XGBoost

In [9]:
from xgboost import XGBRegressor

# Fitting a model
X, y = make_regression()
xgb_reg = XGBRegressor().fit(X, y)

In [58]:
trees = xgb_reg.get_booster().trees_to_dataframe()
trees

,Tree,Node,ID,Feature,Split,Yes,No,Missing,Gain,Cover,Category
0,0,0,0-0,f36,0.315701,0-1,0-2,0-1,1.028901e+06,100.0,NaN
1,0,1,0-1,f30,0.445113,0-3,0-4,0-3,3.146140e+05,68.0,NaN
2,0,2,0-2,f28,-0.290013,0-5,0-6,0-5,1.694598e+05,32.0,NaN
3,0,3,0-3,f31,-0.787639,0-7,0-8,0-7,1.381958e+05,44.0,NaN
4,0,4,0-4,f78,-0.061120,0-9,0-10,0-9,2.289839e+05,24.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2419,95,0,95-0,Leaf,NaN,NaN,NaN,NaN,-9.149608e-07,100.0,NaN
2420,96,0,96-0,Leaf,NaN,NaN,NaN,NaN,-8.979647e-07,100.0,NaN
2421,97,0,97-0,Leaf,NaN,NaN,NaN,NaN,-8.809685e-07,100.0,NaN
2422,98,0,98-0,Leaf,NaN,NaN,NaN,NaN,-8.639723e-07,100.0,NaN


In [14]:
len(trees)

100

In [ ]:
# https://github.com/eli5-org/eli5/blob/69637074ad07cdb0d0aa3a650302a721c9272e4b/eli5/xgboost.py#L239
# https://stackoverflow.com/questions/62844000/is-there-any-way-to-calculate-the-values-at-each-node-in-xgboost-trees

In [54]:
# Regexes
feature_regex = re.compile('.*\:\[f(\d+).*')
threshold_regex = re.compile('.*\:\[f\d+<(\d+\.*\d+)]')
left_regex = re.compile('.*\:\[f\d+<\d+\.*\d+]\syes=(\d+)')
right_regex = re.compile('.*\:\[f\d+<\d+\.*\d+]\syes=\d+,no=(\d+)')

nodes_df = []

for tree_index, tree in enumerate(trees):    
    nodes = [st.strip() for st in trees[0].split('\n\t')]
    for node in nodes:
        
        node_index = node.split(':')[0]
        
        feature = int(feature_regex.search(node).group(1))
        threshold = float(threshold_regex.search(node).group(1))
        left_child = int(left_regex.search(node).group(1))
        right_child = int(right_regex.search(node).group(1))
        
        node_df = {
                'tree_index'        : tree_index,
                'node_depth'        : None,
                'node_index'        : node_index,
                'left_child'        : left_child,
                'right_child'       : right_child,
                'parent_index'      : None,
                'split_feature'     : feature,
                'split_gain'        : None,
                'threshold'         : threshold,
                'decision_type'     : None,
                'missing_direction' : None,
                'missing_type'      : None,
                'value'             : None,
                'weight'            : None,
                'count'             : None
        }
        nodes_df.append(node_df)
nodes_df = pd.DataFrame.from_records(nodes_df)

AttributeError: 'NoneType' object has no attribute 'group'

In [53]:
nodes_df

,tree_index,node_depth,node_index,left_child,right_child,parent_index,split_feature,split_gain,threshold,decision_type,missing_direction,missing_type,value,weight,count
0,0,None,0,1,2,None,17,None,0.216996,None,None,None,None,None,None
1,0,None,1,1,2,None,17,None,0.216996,None,None,None,None,None,None
2,0,None,3,1,2,None,17,None,0.216996,None,None,None,None,None,None
3,0,None,7,1,2,None,17,None,0.216996,None,None,None,None,None,None
4,0,None,15,1,2,None,17,None,0.216996,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5295,99,None,28,1,2,None,17,None,0.216996,None,None,None,None,None,None
5296,99,None,39,1,2,None,17,None,0.216996,None,None,None,None,None,None
5297,99,None,51,1,2,None,17,None,0.216996,None,None,None,None,None,None
5298,99,None,52,1,2,None,17,None,0.216996,None,None,None,None,None,None


In [39]:
st = '0:[f17<0.216996461] yes=1,no=2,missing=1'

# Feature
feature = re.compile('.*\:\[f(\d+).*')
feature.search(st).group(1)

# Threshold
threshold = re.compile('.*\:\[f\d+<(\d+\.*\d+)]')
threshold.search(st).group(1)

# Left 
left_regex = re.compile('.*\:\[f\d+<\d+\.*\d+]\syes=(\d+)')
left_regex.search(st).group(1)

# Right 
right_regex = re.compile('.*\:\[f\d+<\d+\.*\d+]\syes=\d+,no=(\d+)')
right_regex.search(st).group(1)

'2'

In [ ]:
nodes = []
for tree_index, tree in enumerate(sk_trees):
    tree = tree[0].tree_
    decision_type = '<='
    has_children = (tree.children_left != -1) & (tree.children_right != -1)


    df = pd.DataFrame({
        'tree_index'        : [tree_index] * tree.node_count,
        'node_depth'        : sklearn_compute_node_depths(tree),
        'node_index'        : list(range(0, tree.node_count)),
        'left_child'        : tree.children_left,
        'right_child'       : tree.children_right,
        'parent_index'      : sklearn_get_parent_idx(tree),
        'split_feature'     : tree.feature,
        'split_gain'        : None,
        'threshold'         : tree.threshold,
        'decision_type'     : [decision_type if hc else None for hc in has_children ],
        'missing_direction' : None,
        'missing_type'      : None,
        'value'             : tree.value.ravel(),
        'weight'            : None,
        'count'             : tree.n_node_samples 
    })
    nodes.append(df)

## CatBoost

In [ ]:
from catboost import CatBoostRegressor

# Fitting a model
X, y = make_regression()
cb_reg = CatBoostRegressor().fit(X, y)

# Extracting trees
#trees = cb_reg.get_booster().get_dump()
#print(trees)